<a href="https://colab.research.google.com/github/CalumHd/GEE_Training/blob/main/SENSE_Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Authentication and Packages**

In [2]:
import ee
# Trigger authentication
ee.Authenticate()
# Initialise library
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=VFEZJ4q-e0y75Q9jDr207512PkLWG0OuV2cfbmxA7YU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5ObSTb_gIH1KSG_TSu-FKcAOEPwJU8k1IcJTpL2kNWjuR4ZqKaZJY

Successfully saved authorization token.


In [ ]:
!pip install geemap
import geemap;

# **Get data for country boundaries and imagery**

In [8]:
# Create a rectangular ROI covering out region of interest in Egypt
roi = ee.Geometry.Rectangle(
        [22.2, 28.0, 22.8, 28.8])


In [9]:
# Get imagery from MODIS for Hispaniola study area
# Data addresss: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1#description
# MODIS data is the median NDVI/EVI data for every 16 days, the dataset is from the Terra satellite. 

modisNDVI = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI')

modisEVI = ee.ImageCollection('MODIS/006/MOD13Q1').select('EVI')

                              # Ignore the section below
                              #modisGlobal = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(ee.Geometry.Polygon((
                              #        [[[-74.60668900677238, 20.088333497505946],
                              #          [-74.60668900677238, 17.687361057873034],
                              #          [-68.11376908489738, 17.687361057873034],
                              #          [-68.11376908489738, 20.088333497505946]]])
                              #)).filterDate('2019-01-01', '2019-12-31').sort('MODLAND QA Bits').first()

In [10]:
# Get polygons for Dominican Republic and Haiti

# Data source for country polygons: https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# Filter the featurecollection to give just a Haiti polygon
haiti = countries.filter(ee.Filter.eq('country_na', 'Haiti'))
# Filter the featurecollection to give just a Dominican Republic polygon
domRep = countries.filter(ee.Filter.eq('country_na', 'Dominican Republic'))

In [11]:
## Define functions for clipping imagery to the Dominican Republic and Haiti

# Define a function which will take a given image and clip to Haiti
def haitiClip(image):
    return image.clip(haiti)
# Define a fucntion which will take images and clip to Dominican Republic
def domRepClip(image):
    return image.clip(domRep)

# **Calculate Annual Averages of NDVI from 2000-2010**

In [12]:
# Define a function to calculate annual averages for imagery passed in
def annualAverage(year, imageCollection):
    annualImagery = imageCollection.filterDate(str(year) + '-01-01', str(year) + '-12-31')
    return annualImagery.mean()  


In [13]:
# Run function for annnualAverage against every year of MODIS NDVI data since 2000 (clumsy, better if this could be done in a loop)
averageNDVI00 = annualAverage(2000, modisNDVI)
averageNDVI01 = annualAverage(2001, modisNDVI)
averageNDVI02 = annualAverage(2002, modisNDVI)
averageNDVI03 = annualAverage(2003, modisNDVI)
averageNDVI04 = annualAverage(2004, modisNDVI)
averageNDVI05 = annualAverage(2005, modisNDVI)
averageNDVI06 = annualAverage(2006, modisNDVI)
averageNDVI07 = annualAverage(2007, modisNDVI)
averageNDVI08 = annualAverage(2008, modisNDVI)
averageNDVI09 = annualAverage(2009, modisNDVI)
averageNDVI10 = annualAverage(2010, modisNDVI)
averageNDVI11 = annualAverage(2011, modisNDVI)
averageNDVI12 = annualAverage(2012, modisNDVI)
averageNDVI13 = annualAverage(2013, modisNDVI)
averageNDVI14 = annualAverage(2014, modisNDVI)
averageNDVI15 = annualAverage(2015, modisNDVI)
averageNDVI16 = annualAverage(2016, modisNDVI)
averageNDVI17 = annualAverage(2017, modisNDVI)
averageNDVI18 = annualAverage(2018, modisNDVI)
averageNDVI19 = annualAverage(2019, modisNDVI)
averageNDVI20 = annualAverage(2020, modisNDVI)

In [14]:
# Turn the annual averages into Image Collections
annAvNDVI_10s = ee.ImageCollection([averageNDVI11, averageNDVI12, averageNDVI13, 
                                averageNDVI14, averageNDVI15, averageNDVI16, 
                                averageNDVI17, averageNDVI18, averageNDVI19, 
                                averageNDVI20])

annAvNDVI_00s = ee.ImageCollection([averageNDVI00, averageNDVI01, averageNDVI02,
                                    averageNDVI03, averageNDVI04, averageNDVI05,
                                    averageNDVI05, averageNDVI06, averageNDVI07,
                                    averageNDVI08, averageNDVI09, averageNDVI10])

annAvNDVI_ALL = annAvNDVI_00s.merge(annAvNDVI_10s).sort('system:time_start', True)

                                 
                                

In [15]:
# Get info on the image collections
#annAvNDVI_10s.getInfo()

#annAvNDVI_00s.getInfo()

#annAvNDVI_ALL.getInfo()

# Get the dates of the imagery in the collections
range = annAvNDVI_ALL.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
dateRangeMin = ee.Date(range.get('min'))
dateRangeMax = ee.Date(range.get('max'))
print(f'Date range: {dateRangeMin} to {dateRangeMax}')

annAvNDVI_ALL.getInfo()

Date range: ee.Date({
  "functionInvocationValue": {
    "functionName": "Date",
    "arguments": {
      "value": {
        "functionInvocationValue": {
          "functionName": "Dictionary.get",
          "arguments": {
            "dictionary": {
              "functionInvocationValue": {
                "functionName": "Collection.reduceColumns",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.limit",
                      "arguments": {
                        "ascending": {
                          "constantValue": true
                        },
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.merge",
                            "arguments": {
                              "collection1": {
                                "functionInvocationValue": {
        

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
      'type': 'PixelType'},
     'id': 'NDVI'}],
   'properties': {'system:index': '1_0'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
      'type': 'PixelType'},
     'id': 'NDVI'}],
   'properties': {'system:index': '1_1'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
      'type': 'PixelType'},
     'id': 'NDVI'}],
   'properties': {'system:index': '1_2'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
  

In [16]:
# Clip image collections to Haiti
Hai_annAvNDVI = annAvNDVI_ALL.map(haitiClip)
# Clip image collections to Dominican Republic
Dom_annAvNDVI = annAvNDVI_ALL.map(domRepClip)

In [17]:
# Visualise the annual average image collection as a GIF

# Define arguments for animation function parameters.
videoArgs = {
  'dimensions': 1000,
  'region': roi,
  'framesPerSecond': 4,
  'crs': 'EPSG:4326',
  'min': 0,
  'max': 10000,
  'palette': ['000000', '00FF00']
};

# Print the animation to the console as a ui.Thumbnail using the above defined
# arguments. Note that ui.Thumbnail produces an animation when the first input
# is an ee.ImageCollection instead of an ee.Image.
#print(ui.Thumbnail(tempCol, videoArgs));

# Alternatively, print a URL that will produce the animation when accessed.
print(annAvNDVI_ALL.getVideoThumbURL(videoArgs));

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/4d336650ddc365e1ca4465c90bb428e2-4484c242f194f93b6c210ec79ab109ee:getPixels


In [ ]:
# Visualise the entire image collection as a GIF
modisNDVIgif = modisNDVI.filterDate('2015-06-01', '2019-12-31').sort('system:time_start', True)

# Define arguments for animation function parameters.
videoArgs = {
  'dimensions': 768,
  'region': roi,
  'framesPerSecond': 7,
  'crs': 'EPSG:4326',
  'min': 0,
  'max': 10000,
  'palette': ['000000', '00FF00']
};

# Alternatively, print a URL that will produce the animation when accessed.
print(modisNDVIgif.getVideoThumbURL(videoArgs));

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/26b5ea2b9c1de95b6ab2835c503da292-960bd5c37b82729508011cf9ba114ca9:getPixels


In [ ]:
# Visualise the above for Haiti only
haitiModisNDVIgif = modisNDVI.filterDate('2016-06-01', '2019-12-31').sort('system:time_start', True).map(haitiClip)

# Define arguments for animation function parameters.
videoArgs = {
  'dimensions': 768,
  'region': roi,
  'framesPerSecond': 7,
  'crs': 'EPSG:4326',
  'min': 0,
  'max': 10000,
  'palette': ['000000', '00FF00']
};

# Alternatively, print a URL that will produce the animation when accessed.
print(haitiModisNDVIgif.getVideoThumbURL(videoArgs));

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/d372ba25d9a15ca68d862fe1d6e4a072-23f95327fb962539eaf05e1f18bb215f:getPixels


# **Calculating Change in NDVI by comparing annual averages from 2010-2020 with a decadal 2000-2010 average **

In [ ]:
# Set up the 10 year reference period
refDecade = modisNDVI.filterDate('2000-01-01', '2009-12-31').sort('system:time_start', True) # What happens when it is false or true?
computeDecade = modisNDVI.filterDate('2010-01-01', '2020-12-31').sort('system:time_start', True)

# Compute the mean of the reference decade
meanDecade = refDecade.mean()

# Compute anomalies by subtracting the 2000-2010 mean from each image in a
# collection of 2010-2020 images. Copy the date metadata over to the
# computed anomaly images in the new collection.

def subtracting(image):
  return image.subtract(meanDecade).set('system:time_start', image.get('system:time_start'))

series = computeDecade.map(subtracting)
series_sum = series.sum()




In [ ]:
# Calculate the NDVI anomaly for each annual average NDVI from 2010-2020 against the 2000-2010 decadal average. 
change10 = subtracting(averageNDVI10)
change11 = subtracting(averageNDVI11)
change12 = subtracting(averageNDVI12)
change13 = subtracting(averageNDVI13)
change14 = subtracting(averageNDVI14)
change15 = subtracting(averageNDVI15)
change16 = subtracting(averageNDVI16)
change17 = subtracting(averageNDVI17)
change18 = subtracting(averageNDVI18)
change19 = subtracting(averageNDVI19)
change20 = subtracting(averageNDVI20)

In [ ]:
#change11.getInfo()
#map = Map.setCenter(19.002869, -71.083427);
#map = Map.addLayer({'min': -60000, 'max': 60000, 'palette': ['FF0000', '000000', '00FF00']}, 'EVI anomaly')
#map


In [ ]:
# Make a Image Collection from the change images
changeCollection = ee.ImageCollection([change10,change11, change12, change13, change14,
                                  change15, change16, change17, change18,
                                  change19, change20]).sort('system:time_start', True)

In [ ]:
# Get the dates of the imagery in the change collection
range = changeCollection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
dateRangeMin = ee.Date(range.get('min'))
dateRangeMax = ee.Date(range.get('max'))
print(f'Date range: {dateRangeMin} to {dateRangeMax}')

annAvNDVI_ALL.getInfo()

Date range: ee.Date({
  "functionInvocationValue": {
    "functionName": "Date",
    "arguments": {
      "value": {
        "functionInvocationValue": {
          "functionName": "Dictionary.get",
          "arguments": {
            "dictionary": {
              "functionInvocationValue": {
                "functionName": "Collection.reduceColumns",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.limit",
                      "arguments": {
                        "ascending": {
                          "constantValue": true
                        },
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.fromImages",
                            "arguments": {
                              "images": {
                                "arrayValue": {
                     

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
      'type': 'PixelType'},
     'id': 'NDVI'}],
   'properties': {'system:index': '1_0'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
      'type': 'PixelType'},
     'id': 'NDVI'}],
   'properties': {'system:index': '1_1'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
      'type': 'PixelType'},
     'id': 'NDVI'}],
   'properties': {'system:index': '1_2'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0],
     'data_type': {'max': 32767,
      'min': -32768,
      'precision': 'double',
  

In [ ]:
# Make a GIF of the anomaly per year from 2011-2020 when compared with a decadal mean from 2000-2010

# Define arguments for animation function parameters.
videoArgs = {
  'dimensions': 1000,
  'region': roi,
  'framesPerSecond': 2,
  'crs': 'EPSG:4326',
  'min': -6000,
  'max': 6000,
  'palette': ['FF0000', '#FFA500','000000', '#006400', '00FF00']
};


# Alternatively, print a URL that will produce the animation when accessed.
print(changeCollection.getVideoThumbURL(videoArgs));

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/583ea4604d0111cb026a765fb7ab2c38-05b9570010926b3a953dadbf14c27cc3:getPixels


# **Scratch working below this**

In [ ]:
# Sum the change collection
cumulativeChange = changeCollection.sum()

In [ ]:
meanDecade.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'NDVI'}],
 'type': 'Image'}

In [ ]:
# Display the changes in folium
import folium 
!pip install geehydro
import geehydro
map = folium.Map(location=[19.002869, -71.083427],zoom_start= 8) 
map.addLayer(series_sum, {'min': -60000, 'max': 60000, 'palette': ['FF0000', '000000', '00FF00']}, 'EVI anomaly')
#map.addLayer(averageNDVI05, {'min': 0, 'max': 10000, 'palette': ['000000', '00FF00']}, 'EVI anomaly')
folium.LayerControl().add_to(map)
map

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10114 sha256=10a892e80852807eabcd09ff9caf5f88d3f794f40b11d106b1d796377cbbe4e4
  Stored in directory: /root/.cache/pip/wheels/c5/07/67/5fa6e7271b46bbe0acafdc7105bbee27a39ab7132d251d822d
Successfully built geehydro


Although `map()` applies a function to every image in a collection, the function visits every image in the collection independently. 

Suppose you want to compute a cumulative anomaly ($A_t$) at time t from a time series. To obtain a recursively defined series of the form $A_t = f(Image_t, A_{t-1})$, mapping won't work because the function ($f$) depends on the previous result ($A_{t-1}$). 

For example, suppose you want to compute a series of cumulative Normalized Difference Vegetation Index (NDVI) anomaly images relative to a baseline. Let $A_0 = 0$ and $f(Image_t, A_{t-1}) = Image_t + A_{t-1}$ where $A_{t-1}$ is the cumulative anomaly up to time t-1 and $Image_t$ is the anomaly at time t. Use `imageCollection.iterate()` to make this recursively defined ImageCollection. 

In the following example, the function `accumulate()` takes two parameters: an image in the collection, and a list of all the previous outputs. With each call to `iterate()`, the anomaly is added to the running sum and the result is added to the list. The final result is passed to the ImageCollection constructor to get a new sequence of images:

In [ ]:
# Load MODIS EVI (Enhanced Vegetation index) imagery.
collection = ee.ImageCollection('MODIS/006/MYD13A1').select('EVI')

# Define reference conditions from the first 10 years of data
# Sort chronologically in descending order
reference = collection.filterDate('2001-01-01', '2010-12-31').sort('system:time_start', False)

# Compute the mean of the first 10 years.
mean = reference.mean()

# Compute anomalies by subtracting the 2001-2010 mean from each image in a
# collection of 2011-2014 images. Copy the date metadata over to the
# computed anomaly images in the new collection.

def subtracting(image):
  return image.subtract(mean).set('system:time_start', image.get('system:time_start'))

filtered = collection.filterDate('2011-01-01', '2014-12-31')

series = collection.map(subtracting)
series_sum = series.sum()

# Display cumulative anomalies.
# Use folium to visualize the imagery.
map3 = folium.Map(location=[40.2,-100.811],zoom_start=5)
map3.addLayer(series_sum, {'min': -60000, 'max': 60000, 'palette': ['FF0000', '000000', '00FF00']}, 'EVI anomaly')
map3  

In [ ]:
# Display cumulative anomalies.
# Use folium to visualize the imagery.
import folium
import geehydro
map3 = folium.Map(location=[19.002869, -71.083427],zoom_start=8)
map3.addLayer(series_sum, {'min': -60000, 'max': 60000, 'palette': ['FF0000', '000000', '00FF00'], 'opacity':0.7}, 'EVI anomaly')
map3  

In [ ]:
# Define a function to view all the imagery
import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro
#print(points.getInfo())
# Use folium to visualize the imagery.
map = folium.Map(location=[19.002869, -71.083427],zoom_start= 8) 
map.addLayer(Hai_annAvNDVI[1], {'min':0,'max':10000,'bands': ['NDVI'],'palette': ['000000', '66FF00'], 'opacity':1}, 'Hai_annAvNDVI[1]')
folium.LayerControl().add_to(map)
map

TypeError: ignored

In [ ]:
# View the imagery 
import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro
#print(points.getInfo())
# Use folium to visualize the imagery.
map = folium.Map(location=[19.002869, -71.083427],zoom_start= 8) 
map.addLayer(modisGlobal, {'min':0,'max':7000,'bands': ['EVI'],'palette': ['000000', '66FF00'], 'opacity':1}, 'modisGlobal')
folium.LayerControl()
map


NameError: ignored

In [ ]:
#import geemap
#Map = geemap.Map(center=[40, -100], zoom=4)
#Map.addLayer(modisGlobal, {'min':0, 'max':10000, 'bands': ['EVI'], 'palette': ['000000', '66FF00']}, 'modisGlobal')
#Map

In [ ]:

# Load MODIS EVI (Enhanced Vegetation index) imagery.
collection = ee.ImageCollection('MODIS/006/MOD13Q1').select('EVI')

# Define reference conditions from the first 10 years of data
# Sort chronologically in descending order
reference = collection.filterDate('2000-01-01', '2009-12-31').sort('system:time_start', False)

# Compute the mean of the first 10 years.
mean = reference.mean()

# Compute anomalies by subtracting the 2001-2010 mean from each image in a
# collection of 2010-2020 images. Copy the date metadata over to the
# computed anomaly images in the new collection.

def subtracting(image):
  return image.subtract(mean).set('system:time_start', image.get('system:time_start'))

filtered = collection.filterDate('2010-01-01', '2020-12-31')

series = filtered.map(subtracting)
series_sum = series.sum()

# Display cumulative anomalies.
# Use folium to visualize the imagery.
map3 = folium.Map(location=[19.002869, -71.083427],zoom_start=8)
map3.addLayer(series_sum, {'min': -60000, 'max': 60000, 'palette': ['FF0000', '000000', '00FF00'], 'opacity':0.7}, 'EVI anomaly')
map3  

In [ ]:
print(series)

In [ ]:
type(series)